### onedrive내 train data를 가공해 yolov5 trainset을 만들기 위한 notebook
  - input: 다양한 size의 image, x, y, w, h정보 label
  - output: 손글씨 형태만 crop된 이미지

### onedrive & google drive mount

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &    

nohup: appending output to 'nohup.out'


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### generate

In [ ]:
import zipfile
import cv2
import numpy as np
import os
import shutil
import json
import matplotlib.pyplot as plt
import random

In [ ]:
cnt = 0

In [ ]:
source_file = r'/content/onedrive/playdata/final_project/financial/train/financial_ocr_ts2.zip'
ann_file = r'/content/onedrive/playdata/final_project/financial/train/financial_ocr_tl2.zip'

with zipfile.ZipFile(source_file, 'r') as zf:
	zf.extractall('/content')

with zipfile.ZipFile(ann_file, 'r') as zf:
	zf.extractall('/content')

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(os.listdir(r'/content/result/insure/annotations'), test_size=0.2)
cnt = 0
train_list = []
val_list = []

In [ ]:
with zipfile.ZipFile(r'/content/drive/MyDrive/final_project/train.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
  for j in train:
    with open(f'/content/result/insure/annotations/{j}', encoding='UTF8') as f:
      json_object = json.load(f)
    cnt += 1
    img = cv2.imread(f'/content/result/insure/images/{json_object["name"]}', cv2.IMREAD_GRAYSCALE)
    x_size, y_size = img.shape[1], img.shape[0]
    path_img = f'{str(cnt).zfill(5)}.jpg'
    path_txt = f'{str(cnt).zfill(5)}.txt'
    cv2.imwrite(path_img, cv2.resize(img[:x_size], (640, 640), interpolation=cv2.INTER_AREA))
    with open(path_txt, 'w') as w:
      for dic in json_object['annotations'][0]['bbox']:
          x, y, width, height = int(dic['x']), int(dic['y']), int(dic['width']), int(dic['height'])
          print(x, y, width, height)
          if y+height < x_size:
            print(f'{0} {((x+x+width)/2)/x_size:4f} {((y+y+height)/2)/x_size:4f} {width/x_size:4f} {height/x_size:4f}\n')
            w.write(f'{0} {((x+x+width)/2)/x_size:4f} {((y+y+height)/2)/x_size:4f} {width/x_size:4f} {height/x_size:4f}\n')
    new_zip.write(path_txt)
    new_zip.write(path_img)
    train_list.append(path_img)
    os.remove(path_img)
    os.remove(path_txt)

with open('/content/drive/MyDrive/final_project/train.txt', 'w') as w:
  w.write('\n'.join(train_list))

In [ ]:
with zipfile.ZipFile(r'/content/drive/MyDrive/final_project/val.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
  for j in val:
    with open(f'/content/result/insure/annotations/{j}', encoding='UTF8') as f:
      json_object = json.load(f)
    cnt += 1
    img = cv2.imread(f'/content/result/insure/images/{json_object["name"]}', cv2.IMREAD_GRAYSCALE)
    x_size, y_size = img.shape[1], img.shape[0]
    path_img = f'{str(cnt).zfill(5)}.jpg'
    path_txt = f'{str(cnt).zfill(5)}.txt'
    cv2.imwrite(path_img, cv2.resize(img[:x_size], (640, 640), interpolation=cv2.INTER_AREA))
    with open(path_txt, 'w') as w:
      for dic in json_object['annotations'][0]['bbox']:
          x, y, width, height = int(dic['x']), int(dic['y']), int(dic['width']), int(dic['height'])
          if y+height < x_size:
            w.write(f'{0} {((x+width)/2)/x_size:4f} {((y+height)/2)/x_size:4f} {width/x_size:4f} {height/x_size:4f}\n')
    new_zip.write(path_txt)
    new_zip.write(path_img)
    os.remove(path_img)
    os.remove(path_txt)

with open('/content/drive/MyDrive/final_project/val.txt', 'w') as w:
  w.write('\n'.join(val_list))